In [10]:
import os
import gc
import cv2
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from pytorch_toolbelt import losses as L

# Utils
from tqdm.auto import tqdm

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [11]:
class Customize_Model(nn.Module):
    def __init__(self, model_name, classes_num):
        super().__init__()
        self.encoder= torch.load('pretrained/effb0_ex_1_20_21_22.pth').model
        self.encoder.global_pool= nn.Identity()
        self.encoder.classifier= nn.Identity()
        in_feat= 1280
        self.cla= nn.Conv1d(
                        in_channels=in_feat,
                        out_channels=classes_num,
                        kernel_size=1,
                        stride=1,
                        padding=0,
                        bias=True
                    )
        self.att= nn.Conv1d(
                        in_channels=in_feat,
                        out_channels=classes_num,
                        kernel_size=1,
                        stride=1,
                        padding=0,
                        bias=True
                    )
    
    def forward(self, x):
        x= self.encoder(x)
        x= torch.mean(x, dim=2)
        
        att_map= self.att(x).sigmoid()
        x= self.cla(x).softmax(dim=-1)
#         print(att_map)
        out= x*att_map
        out= torch.sum(out, dim=2)
        out= torch.clamp(out, 0, 1)
        
        return out
    
# model= Customize_Model('', 264).cuda()
# x= torch.rand(1,3,128,313).cuda()
# model(x).shape

In [12]:
def get_train_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
#         A.Resize(img_size, img_size),
        
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.Blur(blur_limit= 3, p=0.3), 
        A.GaussNoise(p=0.3),
        A.OneOf([
                A.Cutout(max_h_size=10, max_w_size=16),
                A.CoarseDropout(max_holes=4),
            ], p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.15, rotate_limit= 0,
                                        interpolation=cv2.INTER_LINEAR, border_mode=0, p=0.7),
        ToTensorV2(p=1.0),
    ])


def get_test_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
#         A.Resize(img_size, img_size),
        ToTensorV2(p=1.0),
    ])

In [13]:
from toolbox.audio_aug import *

class Customize_Dataset(Dataset):
    def __init__(self, df, transforms=None, mixup=0):
        self.df = df
        self.transforms = transforms
        self.mixup= mixup
        
    def mixup_aug(self, img_1, mask_1, 
                        img_2, mask_2):
        """
        img: numpy array of shape (height, width,channel)
        mask: numpy array of shape (height, width,channel)
        """
        ## mixup
        weight= np.random.beta(a=0.5, b=0.5)
        img= img_1*weight + img_2*(1-weight)
        mask= mask_1*weight + mask_2*(1-weight)
        return img.astype(np.uint8), mask
    
    def read_data(self, data):
        img = cv2.imread(data['image_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        label= [0]*CFG['num_classes']
        cls = data['label']
        label[cls]= 1
        label= np.array(label)
        return img, label
    
    def __getitem__(self, index):
        data = self.df.loc[index]
        img, label= self.read_data(data)
        
        # use mixup
        if self.mixup and np.random.rand() >= (1-self.mixup):
            img_1= img
            label_1= np.array(label)
            while True:
                indx= np.random.randint(len(self.df))
                data= self.df.loc[indx]
                img_2, label_2= self.read_data(data)
                if label_1.argmax(0)!=label_2.argmax(0): break
            img, label= self.mixup_aug(img_1, label_1, 
                                       img_2, label_2)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': torch.tensor(img/255, dtype=torch.float32),
            'label': torch.tensor(label, dtype=torch.float32),
        }
    
    def __len__(self):
        return len(self.df)

In [14]:
class Customize_loss(nn.Module):
    def  __init__(self):
        super().__init__()
        self.CrossEntropy= nn.CrossEntropyLoss(weight= None, label_smoothing=0.25)
        self.FocalCosineLoss= L.FocalCosineLoss()
        self.soft_ce= L.SoftCrossEntropyLoss(smooth_factor=0.25)
        self.bi_temp= L.BiTemperedLogisticLoss(t1=0.8, t2=1.2)
        self.bce= nn.BCELoss()
        self.nll= nn.NLLLoss()
    
    def forward(self, y_pred, y_true):
        loss= 1.0 * self.bce(y_pred, y_true)
        return loss

In [15]:
def train_epoch(dataloader, model, criterion, optimizer):
    scaler= amp.GradScaler()
    model.train()

    ep_loss= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        
        preds= model(imgs)
        loss= criterion(preds, labels)
        ep_loss.append(loss.item())
        loss/= CFG['gradient_accumulation']
        loss.backward()
            
        if (i+1) % CFG['gradient_accumulation']== 0:
            optimizer.step()
            optimizer.zero_grad()
                
    return np.mean(ep_loss)

In [16]:
from metrics import *

def valid_epoch(dataloader, model, criterion):
    model.eval()
    
    ep_loss= []
    all_pred= []
    all_label= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        all_label.extend(labels.cpu().numpy())
        
        with torch.no_grad():
            preds= model(imgs)
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
        all_pred.extend(preds.cpu().numpy())
        
    
    ## caculate metrics
    all_label= np.array(all_label).argmax(1)
    all_pred= np.array(all_pred)
    
    acc= Accuracy(all_pred, all_label)
    print(f'accuracy: {acc}')
    recall= Mean_Recall(all_pred, all_label)
    print(f'mean_recall: {recall}')
    
    cmap= padded_cmap(all_pred, all_label)
    print(f'cmap: {cmap}')
    
    score= cmap
    return np.mean(ep_loss), score

# CFG

In [17]:
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts.ch_in1k',
 'beit_base_patch16_224.in22k_ft_in22k',
 'beit_base_patch16_224.in22k_ft_in22k_in1k',
 'beit_base_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_224.in22k_ft_in22k',
 'beit_large_patch16_224.in22k_ft_in22k_in1k',
 'beit_large_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_512.in22k_ft_in22k_in1k',
 'beitv2_base_patch16_224.in1k_ft_in22k',
 'beitv2_base_patch16_224.in1k_ft_in22k_in1k',
 'beitv2_large_patch16_224.in1k_ft_in22k',
 'beitv2_large_patch16_224.in1k_ft_in22k_in1k',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_rw_224.sw_in1k',
 'coatnet_1_rw_224.sw_in1k',
 'coatnet_2_rw_224.sw_in12k',
 'coatnet_2_rw_224.sw_in12k_ft_in1k',
 'coatnet_3_rw_224.sw_in12k',
 'coatnet_bn_0_r

In [18]:
CFG= {
    'fold': 0,
    'epoch': 30,
    'model_name': 'tf_efficientnet_b0_ns',
    'finetune': False,
    
    'img_size': 128,
    'batch_size': 64,
    'gradient_accumulation': 1,
    'gradient_checkpoint': False,
    'drop_out': 0.3,
    'drop_path': 0.2,
    
    'lr': 3e-4,
    'weight_decay': 5e-4,
    
    'num_classes': 264,
    'load_model': f"./train_model_copy/cv0_best.pth", #False,
    'save_model': './train_model_copy'
}

if CFG['finetune']:
    CFG['lr']= 3e-5
    CFG['load_model']= f"./train_model_copy/cv{CFG['fold']}_best.pth"
CFG

{'fold': 0,
 'epoch': 30,
 'model_name': 'tf_efficientnet_b0_ns',
 'finetune': False,
 'img_size': 128,
 'batch_size': 64,
 'gradient_accumulation': 1,
 'gradient_checkpoint': False,
 'drop_out': 0.3,
 'drop_path': 0.2,
 'lr': 0.0003,
 'weight_decay': 0.0005,
 'num_classes': 264,
 'load_model': './train_model_copy/cv0_best.pth',
 'save_model': './train_model_copy'}

# Prepare Dataset

In [19]:
df= pd.read_csv('Data/train.csv')
label_name= list(df['label_name'].unique())

df_1= pd.read_csv('Data/train_ex2020.csv')
df_2= pd.read_csv('Data/train_ex2021.csv')
df_3= pd.read_csv('Data/train_ex2022.csv')
df_4= pd.read_csv('Data/train_ex1.csv')
ex_df= pd.concat([df_1, df_2, df_3, df_4], axis=0)
ex_df= ex_df[ex_df['label_name'].isin(label_name)].reset_index(drop=True)

la= list(ex_df['label_name'].unique())
for l in la:
    ex_df.loc[ ex_df['label_name']==l, 'label' ]= label_name.index(l)
ex_df

,image_path,label,label_name,group,fold
0,Data/train_img_ex2020\barswa\XC134349._15804.png,20,barswa,XC134349.,1.0
1,Data/train_img_ex2020\barswa\XC134349._15805.png,20,barswa,XC134349.,1.0
2,Data/train_img_ex2020\barswa\XC134349._15806.png,20,barswa,XC134349.,1.0
3,Data/train_img_ex2020\barswa\XC134349._15807.png,20,barswa,XC134349.,1.0
4,Data/train_img_ex2020\barswa\XC134349._15808.png,20,barswa,XC134349.,1.0
...,...,...,...,...,...
17325,Data/train_img_ex1\greegr\XC58490._95648.png,106,greegr,XC58490.,2.0
17326,Data/train_img_ex1\greegr\XC586205._95649.png,106,greegr,XC586205.,4.0
17327,Data/train_img_ex1\greegr\XC84914._95650.png,106,greegr,XC84914.,2.0
17328,Data/train_img_ex1\greegr\XC84914._95651.png,106,greegr,XC84914.,2.0


In [20]:
df= pd.read_csv('Data/train.csv')

train_dataset= df[df['fold']!=CFG['fold']].reset_index(drop=True)
train_dataset= pd.concat([train_dataset, ex_df],axis=0).reset_index(drop=True)
train_dataset= df[df['fold']!=CFG['fold']].reset_index(drop=True)
label= train_dataset['label'].unique().tolist()
n= 500
for i,g in enumerate(tqdm(label)):
    sample_df= train_dataset[train_dataset['label']==g]
    if len(sample_df)>=500: sample_df= sample_df.sample(n=n, replace=False, random_state=1).reset_index(drop=True)
    elif len(sample_df)<50: sample_df= sample_df.sample(n=50, replace=True, random_state=1).reset_index(drop=True)
    if i==0: new_df= sample_df
    else: new_df= pd.concat([new_df, sample_df], axis=0).reset_index(drop=True)
train_dataset= new_df

valid_dataset= df[df['fold']==CFG['fold']].reset_index(drop=True)
print(f'train dataset: {len(train_dataset)}')
print(f'valid dataset: {len(valid_dataset)}')

train_dataset= Customize_Dataset(train_dataset, get_train_transform(CFG['img_size']), mixup=0.65)
valid_dataset= Customize_Dataset(valid_dataset, get_test_transform(CFG['img_size']), mixup=False)

train_loader= DataLoader(train_dataset, batch_size= CFG['batch_size'], shuffle=True, num_workers=0)
valid_loader= DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=0)
df.head()

  0%|          | 0/264 [00:00<?, ?it/s]

train dataset: 56523
valid dataset: 26194


C:\Users\User\anaconda3\envs\kaggle_smbcd\lib\site-packages\albumentations\augmentations\dropout\cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


,image_path,label,label_name,group,fold
0,Data/train_img\abethr1\XC128013._0.png,0,abethr1,XC128013.,1.0
1,Data/train_img\abethr1\XC128013._1.png,0,abethr1,XC128013.,1.0
2,Data/train_img\abethr1\XC128013._2.png,0,abethr1,XC128013.,1.0
3,Data/train_img\abethr1\XC128013._3.png,0,abethr1,XC128013.,1.0
4,Data/train_img\abethr1\XC128013._4.png,0,abethr1,XC128013.,1.0


# Train

In [21]:
## create model
if CFG['load_model']:
    print(f"load_model: {CFG['load_model']}")
    model= torch.load(CFG['load_model'], map_location= 'cuda')
else:
    model= Customize_Model(CFG['model_name'], CFG['num_classes'])
model.to('cuda')
    
## hyperparameter
criterion= Customize_loss()
optimizer= optim.AdamW(model.parameters(), lr= CFG['lr'], weight_decay= CFG['weight_decay'])

## start training
best_score= 0
for ep in range(1, CFG['epoch']+1):
    print(f'\nep: {ep}')
    
    train_loss= train_epoch(train_loader, model, criterion, optimizer)
    valid_loss, valid_acc= valid_epoch(valid_loader, model, criterion)
    print(f'train loss: {round(train_loss, 5)}')
    print(f'valid loss: {round(valid_loss, 5)}, valid_acc: {round(valid_acc, 5)}')
    
    if valid_acc >= best_score:
        best_score= valid_acc
        torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_best.pth")
        print(f'model save at score: {round(best_score, 5)}')
        
    ## save model every epoch
    torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_ep{ep}.pth")

load_model: ./train_model_copy/cv0_best.pth

ep: 1


  0%|          | 0/884 [00:00<?, ?it/s]

C:\Users\User\AppData\Local\Temp\ipykernel_10796\1559028917.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'image': torch.tensor(img/255, dtype=torch.float32),


  0%|          | 0/819 [00:00<?, ?it/s]

accuracy: 0.7019546461021609
mean_recall: 0.47578966661754085


...

C:\Users\User\AppData\Local\Temp\ipykernel_10796\1559028917.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'image': torch.tensor(img/255, dtype=torch.float32),


  0%|          | 0/819 [00:00<?, ?it/s]

accuracy: 0.7370771932503627
mean_recall: 0.5349779826216502


C:\Users\User\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


cmap: 0.7124046865622629
train loss: 0.00518
valid loss: 0.00818, valid_acc: 0.7124
